In [5]:
import re
import os
import json
from collections import defaultdict

import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [6]:
from pymorphy2 import MorphAnalyzer

In [7]:
REPORTS_TXT_DIRECTORY = 'reports_txt'
REPORTS_TXT_LEMMATIZED_DIRECTORY = 'reports_txt_lemmatized'
REPORTS_TXT_1000 = 'reports_txt_1000'
REPORTS_TXT_100 = 'reports_txt_100'
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+")
stopwords_ru = stopwords.words("russian")

### Лемматизация отчетов

In [8]:
morph = MorphAnalyzer()

In [9]:
def lemmatize(doc):
    doc = EXCLUDE_PATTERNS.sub(' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return ' '.join(tokens)

In [10]:
for report in tqdm(os.listdir(REPORTS_TXT_DIRECTORY)):
    if not report.endswith('.txt'):
            continue
    report_filename = f'{REPORTS_TXT_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lemmatized = lemmatize(report_text)
    _, report = report.rsplit('_', maxsplit=1)
    report_lemmatized_filename = f'{REPORTS_TXT_LEMMATIZED_DIRECTORY}/{report}'
    with open(report_lemmatized_filename, 'w') as f:
        f.write(report_text_lemmatized)

100%|█████████████████████████████████████| 1187/1187 [2:04:42<00:00,  6.30s/it]


In [13]:
len(os.listdir(REPORTS_TXT_LEMMATIZED_DIRECTORY))

1186

### Статистика по словам

In [3]:
rep_words_freq = defaultdict(int) # в скольких отчетах употребляется слово
words_freq = defaultdict(int) # сколько раз всего употребляется слово 

In [4]:
for report in tqdm(os.listdir(REPORTS_TXT_LEMMATIZED_DIRECTORY)):
    if not report.endswith('.txt'):
        continue

    report_filename = f'{REPORTS_TXT_LEMMATIZED_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    
    if len(report_text_lst) < 40:
        os.remove(report_filename)
        continue

    report_words = set()
    for word in report_text_lst:
        words_freq[word] += 1
        if not word in report_words:
            report_words.add(word)
            rep_words_freq[word] += 1

100%|███████████████████████████████████████| 1178/1178 [01:43<00:00, 11.37it/s]


In [5]:
amount_of_reports = len(os.listdir(REPORTS_TXT_LEMMATIZED_DIRECTORY))
amount_of_reports

1178

In [17]:
with open('rep_words_freq.json', 'w') as json_file:
    json.dump(rep_words_freq, json_file, indent=4)
    
with open('words_freq.json', 'w') as json_file:
    json.dump(words_freq, json_file, indent=4)

In [6]:
rep_words_freq_percent = {k: v / amount_of_reports for (k, v) in rep_words_freq.items()}
with open('rep_words_freq_percent.json', 'w') as json_file:
    json.dump(rep_words_freq_percent, json_file, indent=4)

In [12]:
between_2_and_30 = set()
for key, value in tqdm(rep_words_freq_percent.items()):
    if value >= 0.01 and value <= 0.3:
        between_2_and_30.add(key)

100%|██████████████████████████████| 143042/143042 [00:00<00:00, 2546375.14it/s]


In [13]:
len(between_2_and_30)

19854

In [15]:
between_2_and_30

{'задержание',
 'возродить',
 'целе\xad',
 'незави\xad',
 'работни\xad',
 'краснокамский',
 'филиальный',
 'корунд',
 'самодеятельный',
 'венчурный',
 'своеобразный',
 'приехать',
 'природо\xad',
 'роберт',
 'умерший',
 'подчас',
 'убедиться',
 'биоиндикация',
 'гипроспецгаз',
 'пэтф',
 'псо',
 'семейство',
 'мхат',
 'фбуз',
 'технополис',
 'заполярный',
 'ру\xad',
 'дз',
 'излагаться',
 'юрхаровский',
 'смолистый',
 'кузнецов',
 'крюков',
 'теплоэлектроцентраль',
 'гать',
 'предрейсовый',
 'повод',
 'эластомер',
 'ужесточать',
 'торможение',
 'угледобыча',
 'мун',
 'трактовка',
 'акселерационный',
 'фотопроект',
 'выбывать',
 'шамильевич',
 'отборочный',
 'табаков',
 'абк',
 'ванино',
 'быт',
 'пережить',
 'литр',
 'тепломеханический',
 'павлодарский',
 'физлицо',
 'нобелевский',
 'пит',
 'нераспространение',
 'обход',
 'тренный',
 'консалтинг',
 'внаём',
 'аванс',
 'ай',
 'ораторский',
 'мопо',
 'тепловоз',
 'переизбыток',
 'необъективность',
 'политически',
 'предоставль',
 'доброво

In [18]:
for report in tqdm(os.listdir(REPORTS_TXT_LEMMATIZED_DIRECTORY)[:1000]):
    if not report.endswith('.txt'):
        continue
    report_filename = f'{REPORTS_TXT_LEMMATIZED_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word in between_2_and_30]
    new_report_filename = f'{REPORTS_TXT_1000}/{report}'
    with open(new_report_filename, 'w') as f:
        f.write(' '.join(report_words))
        
for report in tqdm(os.listdir(REPORTS_TXT_LEMMATIZED_DIRECTORY)[1000:]):
    if not report.endswith('.txt'):
        continue
    report_filename = f'{REPORTS_TXT_LEMMATIZED_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word in between_2_and_30]
    new_report_filename = f'{REPORTS_TXT_100}/{report}'
    with open(new_report_filename, 'w') as f:
        f.write(' '.join(report_words))

100%|█████████████████████████████████████████| 178/178 [00:14<00:00, 11.92it/s]


### Переходим в kaggle

[блокнот в kaggle](https://www.kaggle.com/code/michaelyashchenko/top2vec)

### Убираем имена компаний

In [1]:
import pandas as pd

In [2]:
rspp_df = pd.read_csv('rspp_reports.csv')
rspp_df.head()

,Unnamed: 0,компания,сектор,год,тип отчета,ссылка на отчет
0,0,РУДН,"Образование, здравоохранение",2021,ОУР,/download/7033aa33d6bfc2f5e069f547a5b15109/
1,1,"ОАО ""МРСК Урала""",Энергетика,2021,ИО,/download/af503e07dd6b861d1ed3048c36868cc9/
2,2,"ПАО ""Россети Сибирь""",Энергетика,2021,ИО,/download/57becde4be827f45bedf2a46f58d793a/
3,3,"ПАО ""Россети Юг""",Энергетика,2021,ИО,/download/851fbaea09387885cefa38bd3b6838f3/
4,4,ПАО «Россети Ленэнерго»,Энергетика,2021,ИО,/download/2034b8c8e84e4bba78049bdd976fc122/


In [20]:
company_names = list(rspp_df['компания'])

In [21]:
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-\«\»]+")

def lemmatize_name(company_name):
    comapny_name = EXCLUDE_PATTERNS.sub('', company_name)
    tokens = set()
    for token in comapny_name.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]    
            tokens.add(token)
    return tokens

In [22]:
lemmatized_company_names = set()
for company_name in tqdm(company_names):
    lemmatized_company_names.update(lemmatize_name(company_name))

100%|█████████████████████████████████████| 1388/1388 [00:01<00:00, 1118.49it/s]


In [23]:
len(lemmatized_company_names)

324

In [24]:
lemmatized_company_names

{'авиакомпания',
 'агентство',
 'ак',
 'акб',
 'акига',
 'алмазэргиэнбанк',
 'алрос',
 'альфабанк',
 'алюминиевый',
 'амвэя',
 'ангарский',
 'анк',
 'ао',
 'архангельский',
 'ас',
 'асэ',
 'атомпроект',
 'атомредметзолото',
 'атомэнергомаш',
 'атомэнергопроект',
 'атомэнергопром',
 'афк',
 'ашан',
 'аэропорт',
 'аэрофлот',
 'балтика',
 'банк',
 'барс',
 'бат',
 'башнефть',
 'бизнес',
 'билайн',
 'биржа',
 'бифид',
 'благотворительный',
 'богословский',
 'бренд',
 'брусника',
 'бумажный',
 'бф',
 'виктория',
 'владивостокский',
 'внипиэт',
 'волга',
 'волгателек',
 'волжский',
 'восток',
 'втб',
 'выксунский',
 'вымпелком',
 'вэб',
 'газ',
 'газпром',
 'газпромбанк',
 'гарантинвест',
 'генерировать',
 'геннадий',
 'гидропресс',
 'гк',
 'гмк',
 'гнц',
 'головной',
 'гольцблат',
 'городской',
 'госкорпорация',
 'государственный',
 'группа',
 'грэс',
 'гтлк',
 'губкин',
 'дания',
 'детский',
 'дивизион',
 'евразийский',
 'еврохим',
 'елена',
 'енисейский',
 'еэс',
 'желдортранс',
 'завод',

In [25]:
with open('cn_usual.txt') as f:
    keep_cn = set(map(lambda w: w.strip(), f.read().split(',')))
keep_cn

{'авиакомпания',
 'агентство',
 'аэропорт',
 'банк',
 'бизнес',
 'биржа',
 'благотворительный',
 'богословский',
 'бренд',
 'брусника',
 'бумажный',
 'газ',
 'городской',
 'госкорпорация',
 'государственный',
 'группа',
 'детский',
 'завод',
 'золото',
 'имя',
 'институт',
 'комбинат',
 'коммерческий',
 'коммунальный',
 'компания',
 'концерн',
 'корпорация',
 'кредитный',
 'кристалл',
 'лаборатория',
 'магнит',
 'международный',
 'межрегиональный',
 'механический',
 'мир',
 'муниципальный',
 'научноисследовательский',
 'недвижимость',
 'нефть',
 'никель',
 'новый',
 'общероссийский',
 'отраслевой',
 'партнёр',
 'пассажирский',
 'полюс',
 'почта',
 'приборостроение',
 'работодатель',
 'распределительный',
 'регион',
 'роза',
 'рыбак',
 'северозапад',
 'сервис',
 'сетевой',
 'солидарность',
 'социальный',
 'союз',
 'специализировать',
 'спина',
 'телекоммуникационный',
 'тело',
 'территориальный',
 'университет',
 'федеральный',
 'фонд',
 'химик',
 'химический',
 'холдинг',
 'хутор',
 'ц

In [26]:
lemmatized_company_names = lemmatized_company_names - keep_cn
len(lemmatized_company_names)

239

In [27]:
lemmatized_company_names

{'ак',
 'акб',
 'акига',
 'алмазэргиэнбанк',
 'алрос',
 'альфабанк',
 'алюминиевый',
 'амвэя',
 'ангарский',
 'анк',
 'ао',
 'архангельский',
 'ас',
 'асэ',
 'атомпроект',
 'атомредметзолото',
 'атомэнергомаш',
 'атомэнергопроект',
 'атомэнергопром',
 'афк',
 'ашан',
 'аэрофлот',
 'балтика',
 'барс',
 'бат',
 'башнефть',
 'билайн',
 'бифид',
 'бф',
 'виктория',
 'владивостокский',
 'внипиэт',
 'волга',
 'волгателек',
 'волжский',
 'восток',
 'втб',
 'выксунский',
 'вымпелком',
 'вэб',
 'газпром',
 'газпромбанк',
 'гарантинвест',
 'генерировать',
 'геннадий',
 'гидропресс',
 'гк',
 'гмк',
 'гнц',
 'головной',
 'гольцблат',
 'грэс',
 'гтлк',
 'губкин',
 'дания',
 'дивизион',
 'евразийский',
 'еврохим',
 'елена',
 'енисейский',
 'еэс',
 'желдортранс',
 'зао',
 'зарубежнефть',
 'ик',
 'илим',
 'имииафрикантовый',
 'инжиниринговый',
 'инккапитал',
 'интер',
 'интернэшнл',
 'информация',
 'иркутск',
 'казаньоргсинтез',
 'казмунайгаз',
 'калининградский',
 'камаз',
 'касперский',
 'кб',
 'кем

In [28]:
for report in tqdm(os.listdir(REPORTS_TXT_1000)):
    if not report.endswith('.txt'):
        continue
    report_filename = f'{REPORTS_TXT_1000}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word not in lemmatized_company_names]
    new_report_filename = f'{REPORTS_TXT_1000}/{report}'
    with open(new_report_filename, 'w') as f:
        f.write(' '.join(report_words))
        
for report in tqdm(os.listdir(REPORTS_TXT_100)):
    if not report.endswith('.txt'):
        continue
    report_filename = f'{REPORTS_TXT_100}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word not in lemmatized_company_names]
    new_report_filename = f'{REPORTS_TXT_100}/{report}'
    with open(new_report_filename, 'w') as f:
        f.write(' '.join(report_words))

100%|█████████████████████████████████████████| 178/178 [00:01<00:00, 93.33it/s]
